In [1]:
# Import from used modules

import inputs_model as im
from pyomo.environ import *
import pandas as pd

#### Model Creation

In [2]:
# Model
model = ConcreteModel()

# Sets:                            
model.Nodes = RangeSet( 0, (im.total_nodes - 1) ) # Node Set
model.Functions = RangeSet( 0, (im.total_functions - 1) ) # Function Set
model.Demands = RangeSet( 0, (im.total_demands - 1)) # Demand Set


# Variables:

# Mapping the virtual flow to the physical flow - of each demand
model.w = Var(  model.Nodes, 
                model.Nodes, 
                model.Functions, 
                model.Functions, 
                model.Demands, 
                within = Binary, 
                initialize = 0)

# Mapping if demand 'd' used function 'k' on server 'i'
model.u = Var(  model.Nodes, 
                model.Functions, 
                model.Demands, 
                within = Binary, 
                initialize = 0)

# Mapping if function 'k' are allocated on server 'i'
model.z = Var(  model.Nodes, 
                model.Functions, 
                within = Binary, 
                initialize = 0)

# Indicating if switch i is active
model.y = Var(  model.Nodes,  
                within = Binary, 
                initialize = 0)

# Indicating if PM i is active
model.x = Var(  model.Nodes,  
                within = Binary, 
                initialize = 0)

# Mapping physical links that were used
model.l = Var(  model.Nodes,
                model.Nodes, 
                within = Binary,  
                initialize = 0)

# Mapping of the total demands that used each physical link.
model.number_flows = Var(   model.Nodes,    
                            model.Nodes,
                            within = NonNegativeIntegers, 
                            initialize = 0)

# Fitness Function
def obj_function(model):
    """
        Calculates the objective function value of the optimization model.    

    Args:
        model: The optimization model that contains the decision variables.

    Returns:
        float: The value of the objective function calculated by the optimization model.
    """
    
    net_on = im.pss * sum(model.y[i] for i in model.Nodes)  
    net_used = 2 * im.pp * sum(model.l[i,j] for i in model.Nodes for j in model.Nodes)

    power_net = net_on + net_used
    
    #  Each function consumes 25% of the power consumption
    pm_on = sum(im.psm * model.x[i] for i in model.Nodes)
    pm_used =  (im.pmm - im.psm) * (4/16) * sum(model.z[i, f] for i in model.Nodes for f in model.Functions)

    power_pm = pm_on + pm_used

    return power_net + power_pm
                         
# Objective Function
model.objective = Objective(    rule = obj_function, 
                                sense = minimize 
                            ) 

#### Restrictions

In [3]:
def set_source(model, d):
    '''
    The first function (k) that demand (d) uses is allocated on the origin server (i)
    '''
    
    # Origin of demand
    source = im.demands[d]['source'] 
    
    # Demand (d) used the source (0) function on the source node. 
    set_source = model.u[   source, 
                            0, 
                            d] == 1 
    
    return set_source

model.C1 = Constraint(  model.Demands, 
                        rule = set_source)

In [4]:
def set_destiny(model, d):
    '''
    The last function (k) that demand (d) uses is allocated on the destination server (i).
    '''
    
    # Demand destination 
    destiny = im.demands[d]['destiny'] 

    # Demand (d) uses the destiny function (7) on the destination node
    set_destination = model.u[  destiny,
                                7,
                                d] == 1

    
    return set_destination

model.C2 = Constraint(  model.Demands, 
                        rule = set_destiny)

In [5]:
def set_used(model, d, k):
    '''
    Demand (d) needs to use all the functions that exist in your chain.
    The variable u will be equal to 1 if the demand (d) you are using function (k) on server (i).
    Demand must use all their functions.
    '''
    
    functions_used = im.demands[d]['service']['functions'][k]

    # if (k) is a function that (d) needs to use
    if functions_used == 1:
        
        # demand (d) needs too use function (k) at some node (i)
        used = sum( model.u[i, k, d] 
                    for i in model.Nodes) == 1
    else:
        used = Constraint.Skip

    return used

model.C3 = Constraint(  model.Demands, 
                        model.Functions, 
                        rule = set_used)

In [6]:
def function_allocation(model, d, i, k):
    '''
    Demand (d) can only use function (k) at node (i) if function (k) is allocated at node (i)
    '''   

    return model.u[i, k, d] <= model.z[i, k]
    
model.C4 = Constraint(  model.Demands, 
                        model.Nodes, 
                        model.Functions, 
                        rule = function_allocation)

In [7]:
def flow_conservation(model, d, i, k, l):
    '''
    Flow Conservation - if there is a virtual flow, this flow needs to be  mapper to the physical flow.
    For every demand (d) that is on server (i) using function (k) - it needs to be directed to server j that will use function (l).
    (k,l) is the virtual flow that represents that when leaving function (k), demand (d) needs to use functions (l).
    '''

    virtual_topology = im.demands[d]['service']['sfc'][k][l]

    if (virtual_topology == 1):

        main_conservation = sum(    model.w[i,j,k,l,d] # The demand (d) that is at node (i) using function (k) - goes to node (j) using function (l)
                                    # (leaving i)
                                    for j in model.Nodes 
                                    if ( (i,j) in im.physical_topology ) ) - \
                                    \
                            sum(    model.w[j,i,k,l,d] # The demand (d) that was at node (j) using function (k) - went to ndoe (i) using function (l).
                                    # (Entering l)
                                    for j in model.Nodes 
                                    if ( (j,i) in im.physical_topology) ) \
                            == model.u[i,k,d] - model.u[i,l,d] # Demand (d) used function (k) at node (i) - demand (d) used function (i) at node (i).
    else:
        main_conservation = Constraint.Skip

        
    return main_conservation

model.C5 = Constraint(  model.Demands, 
                        model.Nodes, 
                        model.Functions, 
                        model.Functions, 
                        rule = flow_conservation)

In [8]:
def loop_in(model, j, d):
    '''
    To prevent a demand (d) from entering an input loop on a server (i).
    In other words, for all virtual flows (k,l) of deman (d) --> Demand can only enter service (i) once.
    '''

    virtual_topology = im.demands[d]['service']['sfc']

    avoid_loops_in = \
        sum(    model.w[i, j, k, l, d] # Demand (d) is entering node (j) (where it will use function l) from node (i) (where it was using function k) 
                for i in model.Nodes 
                for k in model.Functions 
                for l in model.Functions 

                if (((i, j) in im.physical_topology) | ((j, i) in im.physical_topology)) & (virtual_topology[k][l] == 1)) <= 1

    return avoid_loops_in

model.C6 = Constraint(  model.Nodes, 
                        model.Demands, 
                        rule = loop_in)

In [9]:
def loop_out(model, i, d):
    '''
    To prevent a demand (d) from looping out of a server (i).
    In other words, for all virtual flows (k,l) of demand (d) --> Demand can only leave service (i) once.
    '''

    virtual_topology = im.demands[d]['service']['sfc']

    avoid_loops_out = \
        sum(    model.w[i,j,k,l,d] # demand (d) is leaving node (i) (where function (k) was used) to node (j) (use function l)
                for j in model.Nodes 
                for k in model.Functions 
                for l in model.Functions 
                
                if (((i, j) in im.physical_topology) | ((j, i) in im.physical_topology)) & (virtual_topology[k][l] == 1)) <= 1

    return avoid_loops_out 

model.C7 = Constraint(  model.Nodes, 
                        model.Demands, 
                        rule = loop_out)

In [10]:
def total_flow(model, i, j):
    '''
    number_flows is a control variable that presents the number of demands that are using physical link (i,j).
    '''
    
    flow = model.number_flows[i,j] == sum(  model.w[i,j,k,l,d] 
                                            for k in model.Functions 
                                            for l in model.Functions 
                                            for d in model.Demands
                                            )
                
    return flow

model.C8 = Constraint(  model.Nodes, 
                        model.Nodes, 
                        rule = total_flow)

In [11]:
def pm_capacity_control(model, i):
    '''
    
    '''
    
    return sum(im.functions[f][1] * model.z[i, f] for f in model.Functions) <= im.resource_capacity['CPU']

model.C9 = Constraint(  model.Nodes, 
                        rule = pm_capacity_control)

In [12]:
def function_capacity_control(model, i, f):
    '''
    
    '''
    
    return sum(im.demands[d]['service']['bandwidth'] * model.u[i, f, d] for d in model.Demands) <= im.functions[f][2] * model.z[i, f] 

model.C10 = Constraint( model.Nodes,
                        model.Functions,
                        rule = function_capacity_control)

In [13]:
def link_capacity_control(model, i, j):
    '''
    
    '''

    return sum(im.demands[d]['service']['bandwidth'] * model.w[i,j,k,l,d] 
                for k in model.Functions 
                for l in model.Functions 
                for d in model.Demands) <= im.bl 
    

model.C11 = Constraint( model.Nodes,
                        model.Nodes,
                        rule = link_capacity_control)

In [14]:
def delay_control(model, d):
    '''
    
    '''

    return sum(im.functions[f][3] * model.u[i,f,d] for i in model.Nodes for f in model.Functions) + \
           sum(im.physical_topology[(i, j)] * im.link_delay * model.w[i,j,k,l,d] 
               for i in model.Nodes for j in model.Nodes
               for k in model.Functions for l in model.Functions 
               if ( (i,j) in im.physical_topology) ) \
            <= im.demands[0]['service']['delay']
    

model.C12 = Constraint( model.Demands,
                        rule = delay_control)

In [15]:
def link_control(model, i, j):
    '''
    
    '''

    return sum(model.w[i,j,k,l,d] for k in model.Functions for l in model.Functions for d in model.Demands) <= model.l[i, j] * im.bigM
    

model.C13 = Constraint( model.Nodes,
                        model.Nodes,
                        rule = link_control)

In [16]:
def switch_control(model, i):
    '''
    
    '''

    return sum(model.l[i,j] + model.l[j,i] for j in model.Nodes) <= model.y[i] * im.bigM

   

model.C14 = Constraint( model.Nodes,
                        rule = switch_control)

In [17]:
def pm_control(model, i):
    '''
    '''

    return sum(model.z[i, f]  for f in model.Functions) <= model.x[i] * im.bigM

   

model.C15 = Constraint( model.Nodes,
                        rule = pm_control)

#### Execution

In [18]:
# Running Solver
solver = SolverFactory('scip')
results = solver.solve( model, )

#### Outputs

In [19]:
# Cria uma lista com os rotulos dos nodes
nodes = []
for i in model.Nodes:
    nodes.append(f'Node {i}')

# Cria uma lista com os rotulos das demandas
    demands = []
for d in model.Demands:
    demands.append(f'Demanda {d}')

funcoes = ['Source', 'NAT', 'FW', 'TM', 'WOC', 'VOC', 'IDS', 'Destiny']

# DataFrame de Nodes x Funções
df_placement = pd.DataFrame(columns=funcoes, index=nodes)

index = pd.MultiIndex.from_product([demands, nodes], names=['Demanda', 'Servidor'])
# Dataframe de Demanda-Nodes X Funções
df_used = pd.DataFrame(columns=funcoes, index=index)
# Dataframe de Demanda-Nodes X Nodes
df_percurso = pd.DataFrame(columns=nodes, index=index)

In [20]:
tf = sum(value(model.number_flows[:, :]))
p = sum(value(model.z[:, :])) - sum(value(model.z[:, 0])) - sum(value(model.z[:, 7])) 
obj = value(model.objective)


txt = f' A função objetivo é : {obj}  --> Fluxo ({tf}) + Alocação ({p})' 
print(txt)

 A função objetivo é : 3786.0  --> Fluxo (9.0) + Alocação (35.0)


In [21]:
print(f"Funções x Servidores - Onde a função k está sendo alocada? (Variável : X)")
print(f"=" * 60)

for i in model.Nodes:
    n = f'Node {i}'
    for k in model.Functions:
        if value(model.z[i,k]) == 1:
            df_placement.loc[n, funcoes[k]] = value(model.z[i,k])
        else:
            df_placement.loc[n, funcoes[k]] = '.'

df_placement

Funções x Servidores - Onde a função k está sendo alocada? (Variável : X)


,Source,NAT,FW,TM,WOC,VOC,IDS,Destiny
Node 0,1.0,1.0,1.0,1.0,1.0,.,.,.
Node 1,1.0,1.0,1.0,.,.,.,1.0,1.0
Node 2,.,.,.,.,.,.,.,.
Node 3,.,.,.,.,.,.,.,.
Node 4,1.0,1.0,1.0,1.0,1.0,.,.,1.0
Node 5,1.0,1.0,.,1.0,1.0,.,1.0,1.0
Node 6,.,.,.,.,.,.,.,.
Node 7,.,.,1.0,1.0,1.0,.,1.0,1.0
Node 8,.,.,.,.,.,.,.,.
Node 9,.,.,.,.,.,.,.,.


In [22]:
print(f"Onde a demanda d está usando a função k?  (Variável : U)")
print(f"=" * 45)


for d in model.Demands:
    a = f'Demanda {d}'
    for i in model.Nodes:
        n = f'Node {i}'
        for k in model.Functions:
            if value(model.u[i,k,d]) == 1:
                df_used.loc[(a, n), funcoes[k]] = value(model.u[i,k,d])
            else:
                df_used.loc[(a, n), funcoes[k]] = '.'

df_used

Onde a demanda d está usando a função k?  (Variável : U)


Source  NAT   FW   TM  WOC VOC  IDS Destiny
Demanda   Servidor                                            
Demanda 0 Node 0      1.0  1.0  1.0  1.0  1.0   .    .       .
          Node 1        .    .    .    .    .   .  1.0     1.0
          Node 2        .    .    .    .    .   .    .       .
          Node 3        .    .    .    .    .   .    .       .
          Node 4        .    .    .    .    .   .    .       .
...                   ...  ...  ...  ...  ...  ..  ...     ...
Demanda 7 Node 11       .    .    .    .    .   .    .       .
          Node 12       .    .    .    .    .   .    .       .
          Node 13     1.0  1.0    .    .    .   .    .       .
          Node 14       .    .    .    .    .   .    .       .
          Node 15       .    .  1.0  1.0  1.0   .  1.0     1.0

[128 rows x 8 columns]

In [23]:
print(f"Qual o fluxo da demanda d?  (Variável : W)")
print(f"=" * 30)

# Para cada demanda
for d in model.Demands:
    a = f'Demanda {d}'
    # Para cada nó 
    for i in model.Nodes:
        n1 = f'Node {i}'
        # Para cada nó
        for j in model.Nodes:
            n2 = f'Node {j}'

            # A demanda d utilizou o link j,i
            total = sum(value(model.w[i,j,:,:,d]))
            

            # ---------------------------------------------------------------------------
            if  total > 0:
                
                df_percurso.loc[(a, n1), n2] = sum(value(model.w[i,j,:,:,d]))
            else:
                df_percurso.loc[(a, n1), n2] = '.'

            # ---------------------------------------------------------------------------
            if (i == im.demands[d]['source']) & (j == im.demands[d]['source']):
                if total > 0:
                    df_percurso.loc[(a, n1), n2] = 'O1'
                else:
                    df_percurso.loc[(a, n1), n2] = 'O'

            # ---------------------------------------------------------------------------
            if (i == im.demands[d]['destiny']) & (j == im.demands[d]['destiny']):
                if total > 0:
                    df_percurso.loc[(a, n1), n2] = 'D1'
                else:
                    df_percurso.loc[(a, n1), n2] = 'D'

df_percurso

Qual o fluxo da demanda d?  (Variável : W)


Node 0 Node 1 Node 2 Node 3 Node 4 Node 5 Node 6 Node 7  \
Demanda   Servidor                                                           
Demanda 0 Node 0        O    1.0      .      .      .      .      .      .   
          Node 1        .      D      .      .      .      .      .      .   
          Node 2        .      .      .      .      .      .      .      .   
          Node 3        .      .      .      .      .      .      .      .   
          Node 4        .      .      .      .      .      .      .      .   
...                   ...    ...    ...    ...    ...    ...    ...    ...   
Demanda 7 Node 11       .      .      .      .      .      .      .      .   
          Node 12       .      .      .      .      .      .      .      .   
          Node 13       .      .      .      .      .      .      .      .   
          Node 14       .      .      .      .      .      .      .      .   
          Node 15       .      .      .      .      .      .      .      .   

                   Node 8 Node 9 Node 10 Node 11 Node 12 Node 13 Node 14  \
Demanda   Servidor                                                         
Demanda 0 Node 0        .      .       .       .       .       .       .   
          Node 1        .      .       .       .       .       .       .   
          Node 2        .      .       .       .       .       .       .   
          Node 3        .      .       .       .       .       .       .   
          Node 4        .      .       .       .       .       .       .   
...                   ...    ...     ...     ...     ...     ...     ...   
Demanda 7 Node 11       .      .       .       .       .       .       .   
          Node 12       .      .       .       .       .       .       .   
          Node 13       .      .       .       .       .       O       .   
          Node 14       .      .       .       .       .       .       .   
          Node 15       .      .       .       .       .       .       .   

                   Node 15  
Demanda   Servidor          
Demanda 0 Node 0         .  
          Node 1         .  
          Node 2         .  
          Node 3         .  
          Node 4         .  
...                    ...  
Demanda 7 Node 11        .  
          Node 12        .  
          Node 13      1.0  
          Node 14        .  
          Node 15        D  

[128 rows x 16 columns]